In [1]:
import ratransformers

example = "My name is Clara and I live in Berkeley, California."

ratransformer = ratransformers.RATransformer(
    "dslim/bert-base-NER", 
    relation_kinds=['is_value_of_column', 'is_from_same_row'],
)
model = ratransformer.model
tokenizer = ratransformer.tokenizer

In [2]:
# encode 
encoding = tokenizer(example, return_tensors="pt", input_relations=None)

# forward pass
outputs = model(**encoding)

# get labels ids and convert to label tags
labels = outputs.logits.argmax(-1)
tokens_to_labels = [model.config.id2label[label_id.item()] for label_id in labels[0]]

# print tokens with their predicted NER tags
for i, token_i_map in enumerate(encoding['offset_mapping'][0]):
    span = token_i_map.tolist()
    token = example[span[0]:span[1]]
    if token: # skip special tokens
        print(token, tokens_to_labels[i])

My O
name O
is O
Clara B-PER
and O
I O
live O
in O
Berkeley B-LOC
, O
California B-LOC
. O
